In [1]:
# Check correct mutations are inserted

In [2]:
import glob
import pandas as pd
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein
from Bio.Seq import reverse_complement, transcribe, back_transcribe, translate
from datetime import date

In [3]:
today = date.today().isoformat()
company = "elanco"
order = "R2"
fastafiles = {}

In [4]:
seqs = {}
for fasta in glob.glob("*.fasta"):
    with open(fasta, 'r') as f:
        for i in f:
            if(i[0] == ">"):
                key = i.strip()[1:]
                seqs[key] = ""
            else:
                seqs[key] += i.strip()

In [5]:
df = pd.DataFrame.from_dict(seqs,orient='index').reset_index().rename(columns={'index' : 'Name', 0 : 'sequence'})

In [6]:
df["No_bsaI_site"] = ""
df["AA_seq"] = ""

In [7]:
df.head()
preseq = "GTCACGGTCTCA"
postseq = "CGAGACCAGTCA"
for i,j in zip(df.index, df["sequence"]):
    newseq = j.replace(preseq,"")
    newseq = newseq.replace(postseq,"")
    df.iloc[i,df.columns.get_loc("No_bsaI_site")] = newseq
    df.iloc[i,df.columns.get_loc("AA_seq")] = translate(newseq)

In [8]:
with open(today+"_seqs.fasta",'w') as f:
    for i,j in zip(df["Name"],df["AA_seq"]):
        f.write(">"+i+"\n")
        f.write(j+"\n")

In [9]:
df.to_excel(today+"_"+company+"_"+order+".xlsx")